In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pdb
from scipy.stats import chi2_contingency, chi2
from itertools import combinations, product

In [93]:
df = pd.DataFrame(np.zeros(shape=(2000,2)),columns=['x1','x2'])
df.loc[:,'x1'] = np.random.randint(low=1,high=3,size=2000)
df.loc[:,'x2'] = df.x1 * np.random.randint(low=1,high=3,size=2000)
df.loc[:,'x3'] = np.random.randint(low=1,high=5,size=2000)
df.loc[:,'y'] =  df.x1

/tmp/ipykernel_55325/1800480265.py:2: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,'x1'] = np.random.randint(low=1,high=3,size=2000)
/tmp/ipykernel_55325/1800480265.py:3: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,'x2'] = df.x1 * np.random.randint(low=1,high=3,size=2000)


In [94]:
df

,x1,x2,x3,y
0,1,2,1,1
1,2,4,4,2
2,1,2,4,1
3,2,4,3,2
4,1,2,3,1
...,...,...,...,...
1995,1,1,3,1
1996,1,2,1,1
1997,1,1,3,1
1998,1,2,1,1


In [95]:
df.corr()

,x1,x2,x3,y
x1,1.000000,0.695145,0.010019,1.000000
x2,0.695145,1.000000,0.037951,0.695145
x3,0.010019,0.037951,1.000000,0.010019
y,1.000000,0.695145,0.010019,1.000000


In [97]:
# put in wilson hilferty and overall variable selection rules
def remove_empty_cols(t : np.ndarray):
    """ Remove empty columns, modifying t in place """
    cols = t.shape[1]
    empty_cols = [c for c in range(cols) if t[:,c].sum() == 0]
    return np.delete(t, empty_cols, 1)
    
split_vars = df.columns.to_list()
split_vars.remove('y')        
pairs = [*combinations(split_vars,r=2)]
pvalues = {}
one_dof_stats = {}
residuals = df['y'] - df['y'].mean()
roles = {'x1' : 'c','x2': 'c', 'x3' : 'c'}

for pval_idx, (a,b) in enumerate(pairs):
    
    # case: a and b numeric    
    if roles[a] in ['S','n'] and roles[b] in ['S','n']:
        print('case 1 hit')
        chi_squared = np.zeros(shape=(2,4))
        quadrants = list(product(['lt','gt'],['lt','gt']))
        for idx, (ci,cj) in enumerate(quadrants):
            if ci == 'lt':
                left_bool_idx = df[a] <= df[a].median()
            else:
                left_bool_idx = df[a] > df[a].median()
            if cj == 'lt':
                right_bool_idx = df[b] <= df[b].median()
            else:
                right_bool_idx = df[b] > df[b].median()
            chi_squared[0,idx] = (residuals[left_bool_idx & right_bool_idx] <= 0).sum()
            chi_squared[1,idx] = (residuals[left_bool_idx & right_bool_idx] > 0).sum()
        chi_squared = remove_empty_cols(chi_squared)
        
    # case: a and b categoric
    elif roles[a] == roles[b] and roles[a] == 'c':
        print('case 2 hit')
        alev = df[a].unique()
        blev = df[b].unique()
        cat_pairs = list(product(alev,blev))
        chi_squared = np.zeros(shape=(2, len(cat_pairs)))
        for idx, (ci,cj) in enumerate(cat_pairs):
            chi_squared[0,idx] = (residuals[(df[a] == ci) & (df[b] == cj)] < 0).sum()
            chi_squared[1,idx] = (residuals[(df[a] == ci) & (df[b] == cj)] >= 0).sum()
        chi_squared = remove_empty_cols(chi_squared)
        
    # case: one numeric and one categoric
    elif (roles[a] == 'c' and roles[b] in ['S','n']) or (roles[a] in ['S','n'] and roles[b] == 'c'):
        print('case 3 hit')
        if roles[a] != 'c':
            # ensure categorical variable is a
            a, b = b, a
        alev = df[a].unique()
        chi_squared = np.zeros(shape=(2,2*len(alev)))
        groups = list(product(alev,['lt','gt']))
        for idx, (ci,cj) in enumerate(groups):
            if cj == 'lt':
                bool_idx = df[b] <= df[b].median()
            else:
                bool_idx = (df[b] > df[b].median())
            chi_squared[0,idx] = (residuals[(df[a] == ci) & bool_idx] <= 0).sum()
            chi_squared[1,idx] = (residuals[(df[a] == ci) & bool_idx] > 0).sum()
        chi_squared = remove_empty_cols(chi_squared)
        
    else:
        assert False, "We should never reach this line. Likely caused by unhandled variable role."

    res = chi2_contingency(chi_squared)
    dof, stat = res.dof, res.statistic
    one_dof_stat = wilson_hilferty(stat=stat, dof=dof)
    pval_one_dof = pvalue_for_one_dof(stat=one_dof_stat)
    pvalues[(a,b)] = pval_one_dof
    one_dof_stats[(a,b)] = one_dof_stat
    # convert dof to 1-df and get pvalue
    print(f"{a}, {b} dof = {res.dof}, stat = {one_dof_stat} pval = {pval_one_dof}")


case 2 hit
x1, x2 dof = 3, stat = 1922.588128988558 pval = 0.0
case 2 hit
x1, x3 dof = 7, stat = 1838.604916908076 pval = 0.0
case 2 hit
x2, x3 dof = 11, stat = 855.8420731426004 pval = 0.0


In [85]:
pairs, pvalues

([('x1', 'x2'), ('x1', 'x3'), ('x2', 'x3')],
 {('x2', 'x1'): 0.0, ('x1', 'x3'): 0.1799576773779289, ('x2', 'x3'): 0.0})

In [256]:
split_vars

['x1', 'x2', 'x3']

In [22]:
def wilson_hilferty(stat, dof) -> np.float64:
    """ approximately convert chi-squared with dof degrees of freedom to 1 degree of freedom """
    if dof == 1:
        return stat
    w1 = (math.sqrt(2*stat) - math.sqrt(2*dof - 1) + 1)**2 / 2
    temp = 7/9 + math.sqrt(dof)*( (stat/dof)**(1/3) - 1 + 2 / (9 * dof) )
    w2 = max(0, temp ** 3) 

    w = None
    if stat < dof + 10 * math.sqrt(2*dof):
        w = w2
    elif stat >= dof + 10 * math.sqrt(2*dof) and w2 < stat:
        w = (w1 + w2) / 2
    else:
        w = w1
    return w
    
def pvalue_for_one_dof(stat):
    """ pvalue for a 1 dof chi squared statistic """
    return 1 - chi2.cdf(stat, 1)